#### Dataframe Joins and Optimizations
When we are joining dataframes it is a good practice to use the smaller dataframe on the right side as the joins are done from left to right.
However SparkSQL builds in several optimizations while performing joins and other operations on dataframes.
We can check out the execution plan of SparkSQL for any operation using the function "explain()"
An example is given below.

One other point is: when we need tojoin a dataframe which is small enough to fit into a workernode then it is better to "broadcast" this dataframe. When a dataframe is broadcasted then a copy of it is sent to all workernodes. This avoids some amount of shuffling of data when joining it with the larger dataframe as the smaller dataframe is compltely available locally for each partition of the larger dataframe.

The above optimization also is done automatically by SparkSQL's built-in optimizer. However in case it is not done it can be done manually also as shown in the example below.

In [1]:
import findspark
findspark.init("/usr/local/spark")
import pyspark

Import and initiate findspark to begin with.
And import pyspark

Now start SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

In [3]:
# from pyspark.sql import functions

Create DataFrame from data source - csv file

In [4]:
customerDF = spark.read.load("customers.tsv", format="csv", sep="\t", inferSchema="true")
salestxnDF = spark.read.load("salestxns.tsv", format="csv", sep="\t", inferSchema="true")

Different types of operations on DataFrames

In [5]:
customerDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)



In [6]:
salestxnDF.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: integer (nullable = true)



In [7]:
new_names_cDF = ['customer_id','customer_name','customer_city','customer_state','customer_zipcode']

In [8]:
new_names_sDF = ['salestxn_id','category_id','category_name','product_id','product_name','product_price',\
              'product_quantity','customer_id']

In [9]:
cDF2=customerDF.toDF(*new_names_cDF)

In [10]:
sDF2=salestxnDF.toDF(*new_names_sDF)

In [11]:
cDF2.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [12]:
cDF2.show(5)

+-----------+--------------+-------------+--------------+----------------+
|customer_id| customer_name|customer_city|customer_state|customer_zipcode|
+-----------+--------------+-------------+--------------+----------------+
|      11039|   Mary Torres|       Caguas|            PR|             725|
|       5623|    Jose Haley|     Columbus|            OH|           43207|
|       5829|    Mary Smith|      Houston|            TX|           77015|
|       6336|Richard Maddox|       Caguas|            PR|             725|
|       1708|Margaret Booth|    Arlington|            TX|           76010|
+-----------+--------------+-------------+--------------+----------------+
only showing top 5 rows



In [13]:
sDF2.printSchema()

root
 |-- salestxn_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)



In [14]:
sDF2.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+
|salestxn_id|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+
|          1|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|      11599|
|          2|         48|    Water Sports|      1073|Pelican Sunstream...|       199.99|               1|        256|
|          3|         24| Women's Apparel|       502|Nike Men's Dri-FI...|         50.0|               5|        256|
|          4|         18|  Men's Footwear|       403|Nike Men's CJ Eli...|       129.99|               1|        256|
|          5|         40|     Accessories|       897|Team Golf New Eng...|        24.99|               2|       8827|
+-----------+-----------+----------------+----------+---

Joining the dataframes with the smaller one on leftside to check out the execution plan of SpqrkSQL.
 - explain() shows the physical plan while explain(True) shows logical plan, optimized logical plan also along with physical execution plan.

In [15]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id)

In [16]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain()

== Physical Plan ==
*BroadcastHashJoin [customer_id#52], [customer_id#70], Inner, BuildLeft
:- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
:  +- *Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
:     +- *Filter isnotnull(_c0#12)
:        +- *FileScan csv [_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/pyspark/10_df_broardcast_join/customers.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(_c0)], ReadSchema: struct<_c0:int,_c1:string,_c2:string,_c3:string,_c4:int>
+- *Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
   +- *Filter isnotnull(_c7#42)
      +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_

In [17]:
jDF1=cDF2.join(sDF2,cDF2.customer_id==sDF2.customer_id).explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#52 = customer_id#70)
:- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
:  +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv
+- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
   +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv

== Analyzed Logical Plan ==
customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int, salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int
Join Inner, (customer_id#52 = customer_id#70)
:- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _

We can notice that the execution plan is optimized in that customer dataframe (cDF) is broadcasted.

Let us check using smaller dataframe on rightside as well.

In [18]:
jDF2=sDF2.join(cDF2,sDF2.customer_id==cDF2.customer_id)

In [19]:
type(jDF2)

pyspark.sql.dataframe.DataFrame

In [20]:
jDF2.explain()

== Physical Plan ==
*BroadcastHashJoin [customer_id#70], [customer_id#52], Inner, BuildRight
:- *Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- *Filter isnotnull(_c7#42)
:     +- *FileScan csv [_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/home/hduser/pyspark/10_df_broardcast_join/salestxns.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(_c7)], ReadSchema: struct<_c0:int,_c1:int,_c2:string,_c3:int,_c4:string,_c5:double,_c6:int,_c7:int>
+- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
   +- *Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- *Filter isnotnull(_c0#12)
         +

In [21]:
jDF2.explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
   +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2

We can see that the execution plan is the same optimized one.

The lines below show how to mark a dataframe for broadcast in case required to be done manually.

In [22]:
jDF3=sDF2.join(cDF2.hint("broadcast"),sDF2.customer_id==cDF2.customer_id).explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- ResolvedHint isBroadcastable=true
   +- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS sa

In [23]:
jDF3=sDF2.join(cDF2.hint("broadcast"),sDF2.customer_id==cDF2.customer_id)

In [24]:
jDF3.printSchema()

root
 |-- salestxn_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [25]:
jDF3.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|salestxn_id|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|customer_id|customer_name|customer_city|customer_state|customer_zipcode|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|         58|         45|         Fishing|      1004|Field & Stream Sp...|       399.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         59|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         78|          9|Cardio Equipment|       1

In [26]:
# import pyspark.sql.functions

In [27]:
pyspark.sql.functions.broadcast(cDF2)

DataFrame[customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int]

In [28]:
type(pyspark.sql.functions.broadcast(cDF2))

pyspark.sql.dataframe.DataFrame

In [29]:
jDF4=sDF2.join(pyspark.sql.functions.broadcast(cDF2),sDF2.customer_id==cDF2.customer_id)
type(jDF4)

pyspark.sql.dataframe.DataFrame

In [30]:
jDF4.explain(True)

== Parsed Logical Plan ==
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS salestxn_id#63, _c1#36 AS category_id#64, _c2#37 AS category_name#65, _c3#38 AS product_id#66, _c4#39 AS product_name#67, _c5#40 AS product_price#68, _c6#41 AS product_quantity#69, _c7#42 AS customer_id#70]
:  +- Relation[_c0#35,_c1#36,_c2#37,_c3#38,_c4#39,_c5#40,_c6#41,_c7#42] csv
+- ResolvedHint isBroadcastable=true
   +- Project [_c0#12 AS customer_id#52, _c1#13 AS customer_name#53, _c2#14 AS customer_city#54, _c3#15 AS customer_state#55, _c4#16 AS customer_zipcode#56]
      +- Relation[_c0#12,_c1#13,_c2#14,_c3#15,_c4#16] csv

== Analyzed Logical Plan ==
salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int
Join Inner, (customer_id#70 = customer_id#52)
:- Project [_c0#35 AS sa

In [31]:
jDF4.show(5)

+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|salestxn_id|category_id|   category_name|product_id|        product_name|product_price|product_quantity|customer_id|customer_id|customer_name|customer_city|customer_state|customer_zipcode|
+-----------+-----------+----------------+----------+--------------------+-------------+----------------+-----------+-----------+-------------+-------------+--------------+----------------+
|         58|         45|         Fishing|      1004|Field & Stream Sp...|       399.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         59|         43|Camping & Hiking|       957|Diamondback Women...|       299.98|               1|       9488|       9488|   Mary Smith|      Hialeah|            FL|           33012|
|         78|          9|Cardio Equipment|       1

In [44]:
jDF4.printSchema()

root
 |-- salestxn_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [46]:
jDF4.dropDuplicates(['customer_id'])

DataFrame[salestxn_id: int, category_id: int, category_name: string, product_id: int, product_name: string, product_price: double, product_quantity: int, customer_id: int, customer_id: int, customer_name: string, customer_city: string, customer_state: string, customer_zipcode: int]

In [47]:
jDF4.printSchema()

root
 |-- salestxn_id: integer (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- product_price: double (nullable = true)
 |-- product_quantity: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [33]:
jDF4.createOrReplaceTempView("data")

In [35]:
spark.sql("select customer_city, count(salestxn_id) as no_of_sales from data group by customer_city order by no_of_sales desc limit 1").show()

+-------------+-----------+
|customer_city|no_of_sales|
+-------------+-----------+
|       Caguas|       6877|
+-------------+-----------+



In [49]:
spark.sql("select customer_name, sum(product_quantity) as products, sum(product_price*product_quantity) as amount from data group by customer_name order by products desc limit 1").show()

+-------------+--------+-----------------+
|customer_name|products|           amount|
+-------------+--------+-----------------+
|   Mary Smith|    5166|474434.0199999932|
+-------------+--------+-----------------+

